In [ ]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def create(n, m, c, t):
    xmax = 1000
    x = [i for i in range(xmax)]
    y = {}
    for alg in ["sa", "ga"]:
        Y = []
        for i in itertools.count():
            try:
                df = pd.read_csv("src/out/{}_n{}_m{}_c{}_{}_{}.csv".format(alg, n, m, c, t, i))
            except FileNotFoundError as e:
                break
            time = df['time'].values
            score = df['score'].values
            Y.append([np.interp(i, time, score) for i in x])
        y[alg] = np.average(Y, axis=0)
    return x, y["sa"], y["ga"]

def plot(n, m, c, t):
    t_map = {
    "u": "Uniform graph with {} nodes".format(n),
    "c": "Clustered graph with {} nodes, {} clusters".format(n, c),
    "m": "Mixed graph with {} nodes, {} clusters".format(n, c)
    }
    x, y_sa, y_ga = create(n, m, c, t)
    fig = plt.figure()
    plt.plot(x, y_sa, label="SA")
    plt.plot(x, y_ga, label="GA")
    
    # Plot optimal (bruteforce) if it exists
    try:
        df = pd.read_csv("src/out/brute_n{}_m{}_c{}_{}.csv".format(n, m, c, t))
        score = df['score'].values[0]
        plt.plot(x, score, 'r--', label="Optimal")
    except FileNotFoundError as e: pass

    plt.legend()
    plt.title("{}, {} vehicles".format(t_map[t], m))
    plt.xlabel("Time (μs)")
    plt.ylabel("Score");
    fig.savefig("img/n{}_m{}_c{}_{}.png".format(n, m, c, t), dpi=300)

In [ ]:
cases = [
    (10, 3, 3),
    (10, 5, 3),
    (100, 10, 5),
    (500, 50, 25),
    (1000, 10, 100),
    (1000, 100, 100),
    (1000, 200, 100)
]

for case in cases:
    for t in ["u", "c", "m"]:
        plot(*case, t)